In [21]:
import json 
import pandas as pd
import re
import nltk
from nltk.util import ngrams
from collections import Counter

#Strategy 1 scan after the word won:
df = pd.read_json("gg2013.json/gg2013.json")
df2 = df[df['text'].str.contains(" won ") | df['text'].str.contains(" Won ")]
tweets = df2['text'].tolist()
keywords = []
regexp = re.compile(r'[!?.]+(?=$|\s)')

#Parse Keywords
for tweet in tweets:
    words = tweet.lower().split()
    index_won = words.index("won")
    if words[index_won + 1] == "best":
        keyword = "best "
        curr_index = index_won + 2
        while curr_index != len(words):
            if regexp.search(words[curr_index]):
                keyword += words[curr_index]
                break
            keyword += words[curr_index] + " "
            curr_index += 1
        keywords.append(keyword)

#Analyze common phrase occurences
#Longest award phrase is 12 words
def common_phrases(keywords):
    bigrams = []
    trigrams = []
    quadgrams = []
    multigrams = []
    megagrams = []
    for item in keywords:
        clean_string = re.sub('[^A-Za-z0-9]+', ' ', item)
        tokens = nltk.word_tokenize(clean_string)
        grams2 = ngrams(tokens, 2)
        grams3 = ngrams(tokens, 3)
        grams4 = ngrams(tokens, 4)
        grams9 = ngrams(tokens, 9)
        grams12 = ngrams(tokens, 12)
        for gram in grams2:
            bigrams.append(gram)
        for gram in grams3:
            trigrams.append(gram)
        for gram in grams4:
            quadgrams.append(gram)
        for gram in grams9:
            multigrams.append(gram)
        for gram in grams12:
            megagrams.append(gram)

    common2 = Counter(bigrams).most_common()
    common3 = Counter(trigrams).most_common()
    common4 = Counter(quadgrams).most_common()
    common9 = Counter(multigrams).most_common()
    common12 = Counter(megagrams).most_common()

    common2 = filter(lambda x: x[0][0] == "best", common2)
    common3 = filter(lambda x: x[0][0] == "best", common3)
    common4 = filter(lambda x: x[0][0] == "best", common4)
    common9 = filter(lambda x: x[0][0] == "best", common9)
    common12 = filter(lambda x: x[0][0] == "best", common12)

    common2 = list(common2)
    common3 = list(common3)
    common4 = list(common4)
    common9 = list(common9)
    common12 = list(common12)

    combined = common2 + common3 + common4 + common9 + common12
    return combined

strategy1 = common_phrases(keywords)

#Strategy 2 scan before the word won:
df3 = df[df['text'].str.contains(" goes to ")]
tweets = df3['text'].tolist()
keywords2 = []

#Parse Backwards, stop at the word best
for tweet in tweets:
    words = tweet.lower().split()
    index = words.index("to")
    if words[index - 1] == "goes":
        keyword = ""
        curr_index = index - 2
        while curr_index >= 0:
            if curr_index == 0 or words[curr_index] == "best":
                keyword = words[curr_index] + keyword
                break
            keyword = " " + words[curr_index] + keyword
            curr_index -= 1
        keywords2.append(keyword)
        
strategy2 = common_phrases(keywords2)

#Can also count common strings in strings for strategy2
def most_common_beststring(strings):
    best_string = filter(lambda x: "best" in x, strings)
    best_string = list(best_string)
    return Counter(best_string).most_common()

most_common_beststring(keywords2)

[('best motion picture drama', 93),
 ('best supporting actor, motion picture', 48),
 ('best actress in a tv comedy or musical', 29),
 ('best screenplay', 26),
 ('best supporting actress tv series, miniseries, or tv movie', 22),
 ('best original song', 18),
 ('best actor in a miniseries/tv movie', 17),
 ('best actress', 16),
 ('best director', 15),
 ('best actor tv series - comedy or musical', 14),
 ('best actress in a mini-series/tv movie', 13),
 ('best actress in a motion picture drama', 13),
 ('best supporting actress in a motion picture', 12),
 ('best actor in a motion picture drama', 12),
 ('best supporting actress', 10),
 ('best director for motion picture', 10),
 ('best foreign film', 9),
 ('best animated feature film', 9),
 ('best supporting actor', 8),
 ('best director - motion picture', 8),
 ('best original score', 7),
 ('best supporting actor in a tv show, miniseries or tv movie', 7),
 ('best actress in a tv series, drama,', 7),
 ('best tv comedy/musical', 7),
 ('best miniser